In [10]:
!pip install -q transformers torch sentencepiece pandas



In [11]:
from transformers import pipeline
import pandas as pd


In [12]:
models = {
    "BERT": "bert-base-uncased",
    "RoBERTa": "roberta-base",
    "BART": "facebook/bart-base"
}

text_gen_pipelines = {}
fill_mask_pipelines = {}
qa_pipelines = {}

for name, model_id in models.items():
    print(f"Loading pipelines for {name}...")
    
    # Text Generation
    try:
        text_gen_pipelines[name] = pipeline("text-generation", model=model_id)
    except Exception as e:
        text_gen_pipelines[name] = str(e)
    
    # Fill Mask
    try:
        fill_mask_pipelines[name] = pipeline("fill-mask", model=model_id)
    except Exception as e:
        fill_mask_pipelines[name] = str(e)
    
    # Question Answering
    try:
        qa_pipelines[name] = pipeline("question-answering", model=model_id)
    except Exception as e:
        qa_pipelines[name] = str(e)


Loading pipelines for BERT...


If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Loading weights: 100%|██████████| 202/202 [00:00<00:00, 939.16it/s, Materializing param=cls.predictions.transform.dense.weight]                 
BertLMHeadModel LOAD REPORT from: bert-base-uncased
Key                         | Status     |  | 
----------------------------+------------+--+-
cls.seq_relationship.bias   | UNEXPECTED |  | 
cls.seq_relationship.weight | UNEXPECTED |  | 
bert.pooler.dense.weight    | UNEXPECTED |  | 
bert.pooler.dense.bias      | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Loading weights: 100%|██████████| 202/202 [00:00<00:00, 975.60it/s, Materializing param=cls.predictions.transform.dense.weight]                 
BertForMaskedLM LOAD REPORT from: bert-base-uncased
Key                         | Status     |  | 
----------------------------+------------+--+-
cls.seq_relationship.bias   | UN

Loading pipelines for RoBERTa...


If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`
Loading weights: 100%|██████████| 202/202 [00:00<00:00, 903.18it/s, Materializing param=roberta.encoder.layer.11.output.dense.weight]              
RobertaForCausalLM LOAD REPORT from: roberta-base
Key                             | Status     |  | 
--------------------------------+------------+--+-
roberta.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Loading weights: 100%|██████████| 202/202 [00:00<00:00, 986.92it/s, Materializing param=roberta.encoder.layer.11.output.dense.weight]              
RobertaForMaskedLM LOAD REPORT from: roberta-base
Key                             | Status     |  | 
--------------------------------+------------+--+-
roberta.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you

Loading pipelines for BART...


Loading weights: 100%|██████████| 159/159 [00:00<00:00, 993.51it/s, Materializing param=model.decoder.layers.5.self_attn_layer_norm.weight]   
This checkpoint seem corrupted. The tied weights mapping for this model specifies to tie model.decoder.embed_tokens.weight to lm_head.weight, but both are absent from the checkpoint, and we could not find another related tied weight for those keys
BartForCausalLM LOAD REPORT from: facebook/bart-base
Key                                                           | Status     | 
--------------------------------------------------------------+------------+-
encoder.layers.{0, 1, 2, 3, 4, 5}.final_layer_norm.bias       | UNEXPECTED | 
encoder.layers.{0, 1, 2, 3, 4, 5}.self_attn.out_proj.bias     | UNEXPECTED | 
encoder.layers.{0, 1, 2, 3, 4, 5}.fc2.bias                    | UNEXPECTED | 
encoder.layers.{0, 1, 2, 3, 4, 5}.fc1.bias                    | UNEXPECTED | 
encoder.embed_positions.weight                                | UNEXPECTED | 
encoder.la

In [13]:
prompt = "The future of Artificial Intelligence is"

results_exp1 = {}

for name, pipe in text_gen_pipelines.items():
    print(f"\n{name} Text Generation:")
    
    if isinstance(pipe, str):
        print("Failed to load:", pipe)
        results_exp1[name] = "FAILED"
    else:
        try:
            output = pipe(prompt, max_length=50, num_return_sequences=1)
            text = output[0]["generated_text"]
            print(text)
            results_exp1[name] = text
        except Exception as e:
            print("Error during generation:", e)
            results_exp1[name] = "ERROR"


Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



BERT Text Generation:


Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


The future of Artificial Intelligence is......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... the or and -................................................................................................................................................... "... the some some some some some some some some some some some some some some some some some

RoBERTa Text Generation:
The future of Artificial Intelligence is

BART Text Generation:
The future of Artificial Intelligence is prioritieshower moderationOXToy prior

In [14]:
sentence = "The goal of Generative AI is to [MASK] new content."

results_exp2 = {}

for name, pipe in fill_mask_pipelines.items():
    print(f"\n{name} Fill-Mask:")
    
    if isinstance(pipe, str):
        print("Failed to load:", pipe)
        results_exp2[name] = "FAILED"
    else:
        try:
            output = pipe(sentence)
            top_preds = [pred["token_str"].strip() for pred in output[:3]]
            print("Top predictions:", top_preds)
            results_exp2[name] = top_preds
        except Exception as e:
            print("Error during fill-mask:", e)
            results_exp2[name] = "ERROR"



BERT Fill-Mask:
Top predictions: ['create', 'generate', 'produce']

RoBERTa Fill-Mask:
Error during fill-mask: No mask_token (<mask>) found on the input

BART Fill-Mask:
Error during fill-mask: No mask_token (<mask>) found on the input


In [15]:
context = "Generative AI poses significant risks such as hallucinations, bias, and deepfakes."
question = "What are the risks?"

results_exp3 = {}

for name, pipe in qa_pipelines.items():
    print(f"\n{name} Question Answering:")
    
    if isinstance(pipe, str):
        print("Failed to load:", pipe)
        results_exp3[name] = "FAILED"
    else:
        try:
            output = pipe(question=question, context=context)
            answer = output["answer"]
            score = output["score"]
            print("Answer:", answer)
            print("Confidence Score:", round(score, 4))
            results_exp3[name] = (answer, score)
        except Exception as e:
            print("Error during QA:", e)
            results_exp3[name] = "ERROR"



BERT Question Answering:
Answer: deepfakes
Confidence Score: 0.0218

RoBERTa Question Answering:
Answer: deepfakes.
Confidence Score: 0.0091

BART Question Answering:
Answer: , and
Confidence Score: 0.016


In [16]:
data = []

for model in models.keys():
    data.append({
        "Model": model,
        "Text Generation Output": results_exp1.get(model, "N/A"),
        "Fill-Mask Top Predictions": results_exp2.get(model, "N/A"),
        "QA Answer + Score": results_exp3.get(model, "N/A")
    })

df = pd.DataFrame(data)
df


,Model,Text Generation Output,Fill-Mask Top Predictions,QA Answer + Score
0,BERT,The future of Artificial Intelligence is.........,"[create, generate, produce]","(deepfakes, 0.021799192763864994)"
1,RoBERTa,The future of Artificial Intelligence is,ERROR,"(deepfakes., 0.009068168699741364)"
2,BART,The future of Artificial Intelligence is prior...,ERROR,"(, and, 0.01596701890230179)"
